In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
# import pyspark.sql as pysql
import time
from urllib.parse import urlparse
import os
import pandas as pd
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
filename=urlparse(url).path
filename=os.path.basename(filename)
print(filename)
df=spark.read.csv(SparkFiles.get(filename), header=True)
df.show()

home_sales_revised.csv
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|      

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('mytable')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
s='''
select round(avg(price),2)
from mytable
where
bedrooms=4
'''
df=spark.sql(s).show()

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
s='''
select
year(date)
,round(avg(price),2)
from mytable
where
bedrooms=3
and bathrooms=3
group by
year(date)
order by 1
'''
df=spark.sql(s).show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2019|           287287.82|
|      2020|           294204.16|
|      2021|           294211.46|
|      2022|           292725.69|
+----------+--------------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
s='''
select
year(date)
,round(avg(price),2)
from mytable
where
bedrooms=3
and bathrooms=3
and floors=2
and sqft_living>=2000
group by
year(date)
order by 1
'''
df=spark.sql(s).show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2019|           289859.14|
|      2020|           292289.09|
|      2021|           296330.96|
|      2022|           290242.99|
+----------+--------------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
s='''
select
view
from mytable
where
price>=(select avg(price) from mytable where price>=350000)
order by 1 desc
'''
df=spark.sql(s).show()
print("--- %s seconds ---" % (time.time() - start_time))
print('the answer is 99-100')

+----+
|view|
+----+
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
+----+
only showing top 20 rows

--- 0.6023821830749512 seconds ---
the answer is 99-100


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales AS select * from mytable").show()

++
||
++
++



In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()
s='''
select
view
from mytable
where
price>=(select avg(price) from mytable where price>=350000)
order by 1 desc
'''
cached_data =spark.sql(s)
cached_data.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+
|view|
+----+
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
|  99|
+----+
only showing top 20 rows

--- 0.4667842388153076 seconds ---


##### the run time of the temp table and the cached table are about the same, unless I did something wrong

In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
s='select * from mytable'
df=spark.sql(s)
df.write.partitionBy('date_built').parquet('parquet_date_built', mode='overwrite')

Py4JJavaError: An error occurred while calling o74.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputComm

In [ ]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('parquet_date_built')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_mytable')

In [3]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 
start_time = time.time()
s='''
select
view
from mytable
where
price>=(select avg(price) from p_mytable where price>=350000)
order by 1 desc
'''
p_cached_data =spark.sql(s)
print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `mytable` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 4 pos 5;
'Sort [unresolvedordinal(1) DESC NULLS LAST], true
+- 'Project ['view]
   +- 'Filter ('price >= scalar-subquery#0 [])
      :  +- 'Project [unresolvedalias('avg('price), None)]
      :     +- 'Filter ('price >= 350000)
      :        +- 'UnresolvedRelation [p_mytable], [], false
      +- 'UnresolvedRelation [mytable], [], false


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales").show()

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')